In [1]:
import pandas as pd
#Git_url
train_url = "https://github.com/neurone0/explore_data/raw/main/melbourne-housing-snapshot/Data/train.csv"
test_url = "https://github.com/neurone0/explore_data/raw/main/melbourne-housing-snapshot/Data/test.csv"

In [5]:
from sklearn.model_selection import train_test_split

X = pd.read_csv(train_url,index_col= "Id")
test_data = pd.read_csv(test_url,index_col= "Id")

In [6]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0,subset=['SalePrice'],inplace= True)

In [8]:
y = X.SalePrice
X.drop(['SalePrice'],axis =1, inplace= True)


In [9]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [12]:
low_cardinality_cols = [ col for col in X_train_full.columns if X_train_full[col].dtype == ['object'] and X_[col].nunique < 10]

In [13]:
num_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int64','float64']]

In [15]:
# Keep selected columns only
my_cols = low_cardinality_cols + num_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test_data[my_cols].copy()

In [19]:
# One-hot encode the data (to shorten the code, we use pandas)
print(X_train)
X_train = pd.get_dummies(X_train)
print(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
619           20         90.0    11694            9            5       2007   
871           20         60.0     6600            5            5       1962   
93            30         80.0    13360            5            7       1921   
818           20          NaN    13265            8            5       2002   
303           20        118.0    13704            7            5       2001   
...          ...          ...      ...          ...          ...        ...   
764           60         82.0     9430            8            5       1999   
836           20         60.0     9600            4            7       1950   
1217          90         68.0     8930            6            5       1978   
560          120          NaN     3196            7            5       2003   
685           60         58.0    16770            7 

In [21]:
!pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   --- ------------------------------------ 4.7/56.8 MB 33.7 MB/s eta 0:00:02
   ------ --------------------------------- 8.7/56.8 MB 25.1 MB/s eta 0:00:02
   --------- ------------------------------ 12.8/56.8 MB 22.9 MB/s eta 0:00:02
   ----------- ---------------------------- 16.8/56.8 MB 22.0 MB/s eta 0:00:02
   -------------- ------------------------- 20.7/56.8 MB 21.4 MB/s eta 0:00:02
   ----------------- ---------------------- 24.6/56.8 MB 21.1 MB/s eta 0:00:02
   -------------------- ------------------- 28.8/56.8 MB 20.8 MB/s eta 0:00:02
   ----------------------- ---------------- 32.8/56.8 MB 20.6 MB/s eta 0:00:02
   ------------------------- -------------- 36.7/56.8 MB 20.5 MB/s eta 0:00:01
   ---------------------------- ----------- 40.9/56.8 MB 20.4 MB/s eta 0:00:01
   ------------------------------- -------- 44.8/56.8 MB 20.3 MB/s eta 0:00:01
   ---------------------------------- ----- 48.8/56.8 MB 20.2 M

In [22]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor (random_state = 0) # Your code here

# Fit the model
my_model_1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [23]:
# Define the model
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05) # Your code here

# Fit the model
my_model.fit(X_train, y_train)

# Get predictions
predictions = my_model.predict(X_valid)
from sklearn.metrics import mean_absolute_error
# Calculate MAE
mae = mean_absolute_error(predictions, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae)

Mean Absolute Error: 17978.990234375


In [24]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=1)

# Fit the model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 42686.25


XGBoost is a leading software library for working with standard tabular data (the type of data you store in Pandas DataFrames, as opposed to more exotic types of data like images and videos). With careful parameter tuning, you can train highly accurate models.